# Module 6: Intelligent Agents

Build intelligent agents that reason over financial graph data and make informed decisions.

## Learning Objectives
- Create agents with memory and graph reasoning capabilities
- Build tool-based architectures for financial analysis
- Implement complete investment analysis workflows

## Module Structure
1. **Basic Agent** - Memory and simple tools
2. **Graph Reasoning** - Multi-hop analysis patterns
3. **Complete System** - End-to-end financial intelligence

## Setup and Dependencies

First, we'll install the required packages for building intelligent agents with graph database capabilities.

In [ ]:
# Install required packages for intelligent agents
!pip install neo4j pandas matplotlib python-dotenv

## Import Libraries

We'll import the necessary libraries and create a simple mock LLM for demonstration purposes. In production, you would replace this with a real LLM like OpenAI's GPT or Anthropic's Claude.

In [ ]:
# Import essential libraries for agent development
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List, Dict, Any
from dataclasses import dataclass, field
from neo4j import GraphDatabase

## Create Mock LLM for Demonstrations

This simple mock LLM will generate responses based on prompt keywords. In production, replace this with OpenAI, Anthropic, or other LLM APIs.

In [ ]:
class MockLLM:
    """Simple mock LLM for demonstrations"""
    def generate(self, prompt: str) -> str:
        """Generate responses based on prompt keywords"""
        if 'apple' in prompt.lower():
            return "Apple shows strong fundamentals. Recommended as solid tech investment with moderate risk."
        elif 'risk' in prompt.lower():
            return "Key risks: competition, regulation, supply chain. Overall moderate risk level."
        return "Analysis indicates positive trends with areas requiring monitoring."

# Initialize the mock LLM
llm = MockLLM()
print("✅ Mock LLM initialized")

## Neo4j Database Connection

Every notebook needs to establish a connection to Neo4j. We'll load environment variables and create a database driver with proper error handling.

In [ ]:
# Load environment variables for database connection
from dotenv import load_dotenv
load_dotenv()

# Set up Neo4j connection parameters
NEO4J_URI = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME', 'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD', 'password')

print(f"Connecting to Neo4j at {NEO4J_URI}")

In [ ]:
# Create Neo4j driver and helper function
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def run_query(query, parameters=None):
    """Execute a Cypher query and return results as a list of dictionaries"""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

In [ ]:
# Test the database connection
try:
    result = run_query("RETURN 'Connected to Neo4j!' as message")
    print(f"✅ {result[0]['message']}")
except Exception as e:
    print(f"❌ Neo4j connection failed: {e}")
    print("Please ensure Neo4j is running and credentials are correct")

## Sample Financial Data Creation

Now we'll create a simple financial network in Neo4j for our agents to analyze. This includes companies, risk factors, and relationships between them.

In [ ]:
# Clear any existing data to start fresh
run_query("MATCH (n) DETACH DELETE n")
print("🧹 Cleared existing data")

In [ ]:
# Create company nodes with basic properties
companies = [
    {'name': 'Apple Inc', 'ticker': 'AAPL', 'sector': 'Technology'},
    {'name': 'Microsoft Corp', 'ticker': 'MSFT', 'sector': 'Technology'}
]

for company in companies:
    run_query("CREATE (c:Company $props)", {'props': company})

print("🏢 Created company nodes")

In [ ]:
# Create risk factor nodes
run_query("CREATE (r:RiskFactor {name: 'Market Competition', severity: 'Medium'})")
run_query("CREATE (r:RiskFactor {name: 'Regulatory Changes', severity: 'High'})")
print("⚠️ Created risk factors")

In [ ]:
# Create relationships between companies and risk factors
run_query("""
MATCH (c:Company {name: 'Apple Inc'}), (r:RiskFactor {name: 'Market Competition'})
CREATE (c)-[:FACES_RISK]->(r)
""")

# Create competitive relationship between companies
run_query("""
MATCH (c1:Company {name: 'Apple Inc'}), (c2:Company {name: 'Microsoft Corp'})
CREATE (c1)-[:COMPETES_WITH]->(c2)
""")

print("🔗 Created relationships")
print("✅ Financial data network ready")

## Lesson 1: Basic Financial Agent

Now we'll build our first intelligent agent. This agent will have memory to track conversations and a simple tool to query company data from our graph database.

### Agent Memory System

First, let's create a simple memory system that can track conversations and entities (like companies) that the agent analyzes.

In [ ]:
@dataclass
class AgentMemory:
    """Simple memory to track conversations and entities"""
    conversations: List[Dict] = field(default_factory=list)
    entities: Dict[str, List[str]] = field(default_factory=dict)
    
    def add_conversation(self, query: str, response: str):
        """Add a conversation to memory, keeping only the last 5"""
        self.conversations.append({
            'query': query, 'response': response, 'time': datetime.now()
        })
        if len(self.conversations) > 5:  # Keep memory manageable
            self.conversations.pop(0)
    
    def track_entity(self, entity: str, context: str):
        """Track when entities (like companies) are mentioned"""
        if entity not in self.entities:
            self.entities[entity] = []
        self.entities[entity].append(context)

print("✅ Agent memory system defined")

### Company Data Tool

Next, we'll create a tool that can query our graph database to retrieve company information, risks, and competitors.

In [ ]:
class CompanyTool:
    """Tool to retrieve company information from Neo4j"""
    
    def execute(self, company_name: str) -> Dict[str, Any]:
        """Query the graph for company data and related information"""
        query = """
        MATCH (c:Company {name: $name})
        OPTIONAL MATCH (c)-[:FACES_RISK]->(r:RiskFactor)
        OPTIONAL MATCH (c)-[:COMPETES_WITH]->(comp:Company)
        RETURN c.name as name, c.ticker as ticker, c.sector as sector,
               collect(DISTINCT r.name) as risks,
               collect(DISTINCT comp.name) as competitors
        """
        results = run_query(query, {'name': company_name})
        
        if results:
            data = results[0]
            return {
                'success': True,
                'company': data['name'],
                'ticker': data['ticker'],
                'sector': data['sector'],
                'risks': [r for r in data['risks'] if r],  # Remove empty values
                'competitors': [c for c in data['competitors'] if c]
            }
        return {'success': False, 'error': f'Company {company_name} not found'}

print("✅ Company data tool created")

### Basic Financial Agent

Now let's combine the memory system and tool into a complete financial agent that can analyze companies.

In [ ]:
class FinancialAgent:
    """A basic financial analysis agent with memory and tools"""
    
    def __init__(self, name: str):
        self.name = name
        self.memory = AgentMemory()
        self.company_tool = CompanyTool()
        self.query_count = 0
    
    def get_status(self):
        """Get current agent performance metrics"""
        return {
            'name': self.name,
            'queries': self.query_count,
            'conversations': len(self.memory.conversations),
            'entities_tracked': len(self.memory.entities)
        }

print("✅ Basic agent class defined")

In [ ]:
def analyze_company(self, company_name: str) -> str:
    """Analyze a company using our tool and LLM reasoning"""
    self.query_count += 1
    
    # Step 1: Get company data using our tool
    company_data = self.company_tool.execute(company_name)
    
    if not company_data['success']:
        response = f"Could not find data for {company_name}"
        self.memory.add_conversation(f"Analyze {company_name}", response)
        return response
    
    # Step 2: Create prompt for LLM analysis
    prompt = f"""
    Analyze {company_data['company']} ({company_data['ticker']}) in {company_data['sector']} sector.
    Risks: {', '.join(company_data['risks']) if company_data['risks'] else 'None'}
    Competitors: {', '.join(company_data['competitors']) if company_data['competitors'] else 'None'}
    Provide investment recommendation.
    """
    
    # Step 3: Get LLM analysis
    analysis = llm.generate(prompt)
    
    # Step 4: Store results in memory for future reference
    self.memory.add_conversation(f"Analyze {company_name}", analysis)
    self.memory.track_entity(company_name, f"Analysis on {datetime.now().date()}")
    
    return analysis

# Add the method to our FinancialAgent class
FinancialAgent.analyze_company = analyze_company

print("✅ Company analysis method added to agent")

In [ ]:
# Create our first financial agent
agent = FinancialAgent("InvestmentAnalyst")
print("✅ Basic financial agent created successfully")
print(f"Agent status: {agent.get_status()}")

### Test the Basic Agent

Let's test our basic agent by analyzing some companies and checking its memory.

In [ ]:
print("=== TESTING BASIC AGENT ===\n")

# Test 1: Analyze Apple
print("📊 Analyzing Apple Inc:")
apple_analysis = agent.analyze_company("Apple Inc")
print(f"{apple_analysis}\n")

In [ ]:
# Test 2: Analyze Microsoft
print("📊 Analyzing Microsoft Corp:")
microsoft_analysis = agent.analyze_company("Microsoft Corp")
print(f"{microsoft_analysis}\n")

In [ ]:
# Test 3: Check agent memory and status
print("🧠 Agent Memory:")
print(f"  Conversations: {len(agent.memory.conversations)}")
print(f"  Entities tracked: {list(agent.memory.entities.keys())}")

status = agent.get_status()
print(f"\n📈 Agent Status: {status['queries']} queries, {status['entities_tracked']} entities tracked")

## Lesson 2: Graph Reasoning Agent

Now we'll enhance our agent with graph-based reasoning capabilities. This allows the agent to analyze competitive relationships and risk networks.

### Graph Reasoning Tool

This tool performs two types of graph-based analysis: competitive analysis and risk network analysis.

In [ ]:
class GraphReasoningTool:
    """Tool for graph-based reasoning patterns"""
    
    def competitive_analysis(self, company_name: str):
        """Analyze competitive relationships and market positioning"""
        query = """
        MATCH (c:Company {name: $name})
        OPTIONAL MATCH (c)-[:COMPETES_WITH]->(competitor:Company)
        OPTIONAL MATCH (c)-[:FACES_RISK]->(risk:RiskFactor)<-[:FACES_RISK]-(peer:Company)
        WHERE peer <> c
        RETURN c.name as company, c.sector as sector,
               collect(DISTINCT competitor.name) as direct_competitors,
               collect(DISTINCT peer.name) as risk_peers
        """
        results = run_query(query, {'name': company_name})
        
        if results:
            data = results[0]
            competitors = [c for c in data['direct_competitors'] if c]
            peers = [p for p in data['risk_peers'] if p]
            
            return {
                'success': True,
                'company': data['company'],
                'sector': data['sector'],
                'direct_competitors': competitors,
                'risk_peers': peers,
                'competitive_intensity': 'High' if len(competitors) > 2 else 'Medium' if len(competitors) > 0 else 'Low'
            }
        return {'success': False, 'error': f'Company {company_name} not found'}

print("✅ Competitive analysis method created")

In [ ]:
def risk_analysis(self, company_name: str):
    """Analyze risk exposure through network effects"""
    query = """
    MATCH (c:Company {name: $name})
    OPTIONAL MATCH (c)-[:FACES_RISK]->(risk:RiskFactor)
    OPTIONAL MATCH (c)-[:FACES_RISK]->(shared_risk:RiskFactor)<-[:FACES_RISK]-(other:Company)
    WHERE other <> c
    RETURN c.name as company,
           collect(DISTINCT risk.name) as direct_risks,
           collect(DISTINCT {risk: shared_risk.name, shared_with: other.name}) as network_risks
    """
    results = run_query(query, {'name': company_name})
    
    if results:
        data = results[0]
        direct_risks = data['direct_risks']
        network_risks = [nr for nr in data['network_risks'] if nr['risk']]
        
        risk_score = len(direct_risks) * 20 + len(network_risks) * 10
        risk_level = 'High' if risk_score >= 60 else 'Medium' if risk_score >= 30 else 'Low'
        
        return {
            'success': True,
            'company': data['company'],
            'direct_risks': direct_risks,
            'network_risks': network_risks,
            'risk_score': min(risk_score, 100),
            'risk_level': risk_level
        }
    return {'success': False, 'error': f'Company {company_name} not found'}

# Add the method to our GraphReasoningTool class
GraphReasoningTool.risk_analysis = risk_analysis

print("✅ Risk analysis method added")

### Enhanced Financial Agent

Now we'll create an enhanced agent that combines basic analysis with graph reasoning capabilities.

In [ ]:
class EnhancedAgent(FinancialAgent):
    """Enhanced agent with graph reasoning capabilities"""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.graph_tool = GraphReasoningTool()

print("✅ Enhanced agent class defined")

In [ ]:
def comprehensive_analysis(self, company_name: str):
    """Perform comprehensive company analysis using multiple tools"""
    # Step 1: Get basic company data
    company_data = self.company_tool.execute(company_name)
    if not company_data['success']:
        return {'error': f'Company {company_name} not found'}
    
    # Step 2: Analyze competitive landscape
    competitive_analysis = self.graph_tool.competitive_analysis(company_name)
    
    # Step 3: Analyze risk network
    risk_analysis = self.graph_tool.risk_analysis(company_name)
    
    # Step 4: Generate comprehensive recommendation
    prompt = f"""
    Investment analysis for {company_name}:
    
    Basic: {company_data['sector']} sector company
    Competition: {competitive_analysis['competitive_intensity']} intensity, {len(competitive_analysis['direct_competitors'])} direct competitors
    Risk: {risk_analysis['risk_level']} risk level, score {risk_analysis['risk_score']}/100
    
    Provide BUY/HOLD/SELL recommendation with reasoning.
    """
    
    recommendation = llm.generate(prompt)
    
    # Store in memory
    self.memory.add_conversation(f"Full analysis {company_name}", recommendation)
    self.memory.track_entity(company_name, f"Comprehensive analysis")
    self.query_count += 1
    
    return {
        'company': company_name,
        'basic_data': company_data,
        'competitive_analysis': competitive_analysis,
        'risk_analysis': risk_analysis,
        'recommendation': recommendation
    }

# Add the method to our EnhancedAgent class
EnhancedAgent.comprehensive_analysis = comprehensive_analysis

print("✅ Comprehensive analysis method added")

In [ ]:
# Create enhanced agent with graph reasoning
enhanced_agent = EnhancedAgent("GraphAnalyst")
print("✅ Enhanced agent with graph reasoning created")

### Test Graph-Based Reasoning

Let's test the enhanced agent's comprehensive analysis capabilities.

In [ ]:
print("=== TESTING GRAPH-BASED REASONING ===\n")

# Comprehensive analysis of Apple
analysis = enhanced_agent.comprehensive_analysis("Apple Inc")

print(f"📊 COMPREHENSIVE ANALYSIS: {analysis['company']}")
print(f"Sector: {analysis['basic_data']['sector']}")
print(f"Competitive Intensity: {analysis['competitive_analysis']['competitive_intensity']}")
print(f"Risk Level: {analysis['risk_analysis']['risk_level']} ({analysis['risk_analysis']['risk_score']}/100)")
print(f"\n💼 Recommendation:\n{analysis['recommendation']}")

In [ ]:
# Check enhanced agent status
print(f"\n📈 Enhanced Agent Status: {enhanced_agent.get_status()}")

## Lesson 3: Complete Financial Intelligence System

Now we'll create a complete financial intelligence system that includes market data integration, investment scoring, and portfolio management.

### Market Data Tool

This tool simulates getting market data from external sources. In production, you would connect to real financial APIs.

In [ ]:
class MarketDataTool:
    """Tool to simulate getting market data from external sources"""
    
    def __init__(self):
        # Mock market data - in production, connect to real APIs
        self.mock_data = {
            'AAPL': {'price': 185.25, 'change_percent': 1.29, 'pe_ratio': 28.5},
            'MSFT': {'price': 415.30, 'change_percent': -0.44, 'pe_ratio': 32.1}
        }
    
    def get_data(self, ticker: str):
        """Get market data for a ticker symbol"""
        if ticker in self.mock_data:
            return {'success': True, 'data': self.mock_data[ticker]}
        return {'success': False, 'error': f'No data for {ticker}'}

print("✅ Market data tool created")

### Financial Intelligence System

This system combines all our tools to provide complete investment analysis and portfolio management.

In [ ]:
class FinancialIntelligenceSystem:
    """Complete system for financial intelligence with portfolio management"""
    
    def __init__(self):
        self.agent = EnhancedAgent("FinancialIntelligence")
        self.market_tool = MarketDataTool()
        self.portfolio = {}
        self.alerts = []

print("✅ Financial intelligence system class defined")

In [ ]:
def analyze_investment(self, company_name: str, ticker: str):
    """Complete investment analysis combining graph analysis and market data"""
    # Get comprehensive analysis from our enhanced agent
    analysis = self.agent.comprehensive_analysis(company_name)
    if 'error' in analysis:
        return analysis
    
    # Get current market data
    market_data = self.market_tool.get_data(ticker)
    
    # Calculate investment score (0-100)
    score = 50  # Base score
    
    # Adjust based on market data
    if market_data['success']:
        data = market_data['data']
        # Price momentum
        if data['change_percent'] > 0:
            score += min(data['change_percent'] * 3, 15)
        # P/E ratio evaluation
        if 15 <= data['pe_ratio'] <= 25:
            score += 20
        elif 10 <= data['pe_ratio'] <= 30:
            score += 10
    
    # Adjust based on risk
    risk_score = analysis['risk_analysis']['risk_score']
    score -= risk_score * 0.3
    
    # Adjust based on competitive position
    intensity = analysis['competitive_analysis']['competitive_intensity']
    if intensity == 'Low':
        score += 15
    elif intensity == 'Medium':
        score += 10
    
    investment_score = max(min(score, 100), 0)
    
    return {
        'company': company_name,
        'ticker': ticker,
        'investment_score': investment_score,
        'market_data': market_data.get('data', {}),
        'analysis': analysis,
        'recommendation': 'BUY' if investment_score >= 70 else 'HOLD' if investment_score >= 50 else 'SELL'
    }

# Add the method to our FinancialIntelligenceSystem class
FinancialIntelligenceSystem.analyze_investment = analyze_investment

print("✅ Investment analysis method added")

In [ ]:
def add_to_portfolio(self, company_name: str, ticker: str):
    """Add a company to the monitored portfolio"""
    analysis = self.analyze_investment(company_name, ticker)
    
    self.portfolio[ticker] = {
        'company': company_name,
        'analysis': analysis,
        'added_date': datetime.now()
    }
    
    # Generate alert if investment score is low
    if analysis['investment_score'] < 40:
        self.alerts.append({
            'type': 'low_score',
            'message': f"{company_name} has low investment score: {analysis['investment_score']:.1f}",
            'timestamp': datetime.now()
        })
    
    return analysis

def get_portfolio_summary(self):
    """Get summary of current portfolio"""
    if not self.portfolio:
        return {'message': 'Portfolio is empty'}
    
    scores = [info['analysis']['investment_score'] for info in self.portfolio.values()]
    
    return {
        'total_companies': len(self.portfolio),
        'average_score': sum(scores) / len(scores),
        'companies': [
            {
                'company': info['company'],
                'ticker': ticker,
                'score': info['analysis']['investment_score'],
                'recommendation': info['analysis']['recommendation']
            }
            for ticker, info in sorted(self.portfolio.items(), 
                                     key=lambda x: x[1]['analysis']['investment_score'], 
                                     reverse=True)
        ]
    }

# Add the methods to our FinancialIntelligenceSystem class
FinancialIntelligenceSystem.add_to_portfolio = add_to_portfolio
FinancialIntelligenceSystem.get_portfolio_summary = get_portfolio_summary

print("✅ Portfolio management methods added")

In [ ]:
# Create complete financial intelligence system
intelligence_system = FinancialIntelligenceSystem()
print("✅ Complete financial intelligence system created")

### Test Complete System

Let's test the complete system by analyzing companies and building a portfolio.

In [ ]:
print("=== TESTING COMPLETE FINANCIAL INTELLIGENCE SYSTEM ===\n")

# Analyze and add companies to portfolio
companies = [("Apple Inc", "AAPL"), ("Microsoft Corp", "MSFT")]

for company, ticker in companies:
    print(f"📊 Analyzing {company} ({ticker})")
    analysis = intelligence_system.add_to_portfolio(company, ticker)
    print(f"   Score: {analysis['investment_score']:.1f}/100")
    print(f"   Recommendation: {analysis['recommendation']}")
    print()

In [ ]:
# Display portfolio summary
summary = intelligence_system.get_portfolio_summary()
print("📈 PORTFOLIO SUMMARY")
print(f"Companies: {summary['total_companies']}")
print(f"Average Score: {summary['average_score']:.1f}/100")

print("\n🏆 Rankings:")
for i, company in enumerate(summary['companies'], 1):
    print(f"  {i}. {company['company']} ({company['ticker']}): {company['score']:.1f} - {company['recommendation']}")

In [ ]:
# Check for system alerts
if intelligence_system.alerts:
    print(f"\n⚠️ Alerts: {len(intelligence_system.alerts)}")
    for alert in intelligence_system.alerts:
        print(f"  • {alert['message']}")
else:
    print("\n✅ No alerts generated")

# Display system statistics
print(f"\n📊 System Stats:")
agent_status = intelligence_system.agent.get_status()
print(f"  Queries: {agent_status['queries']}")
print(f"  Entities Tracked: {agent_status['entities_tracked']}")
print(f"  Portfolio Companies: {len(intelligence_system.portfolio)}")

### Simple Visualization Dashboard

Let's create a simple dashboard to visualize our portfolio performance and agent activity.

In [ ]:
def create_dashboard():
    """Create a simple dashboard showing system performance"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Portfolio scores visualization
    if intelligence_system.portfolio:
        companies = []
        scores = []
        colors = []
        
        for ticker, info in intelligence_system.portfolio.items():
            companies.append(f"{info['company'][:10]}\n({ticker})")
            score = info['analysis']['investment_score']
            scores.append(score)
            colors.append('green' if score >= 70 else 'orange' if score >= 50 else 'red')
        
        bars = ax1.bar(companies, scores, color=colors)
        ax1.set_title('Portfolio Investment Scores')
        ax1.set_ylabel('Score (0-100)')
        ax1.set_ylim(0, 100)
        
        # Add score labels on bars
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
                    f'{score:.0f}', ha='center', va='bottom', fontweight='bold')
    else:
        ax1.text(0.5, 0.5, 'No Portfolio Data', ha='center', va='center')
        ax1.set_title('Portfolio Investment Scores')
    
    # Agent activity timeline
    conversations = intelligence_system.agent.memory.conversations
    if conversations:
        times = [conv['time'] for conv in conversations]
        counts = list(range(1, len(times) + 1))
        
        ax2.plot(times, counts, marker='o', linewidth=2)
        ax2.set_title('Agent Activity Timeline')
        ax2.set_ylabel('Cumulative Conversations')
        ax2.tick_params(axis='x', rotation=45)
    else:
        ax2.text(0.5, 0.5, 'No Activity Data', ha='center', va='center')
        ax2.set_title('Agent Activity Timeline')
    
    plt.tight_layout()
    plt.show()

# Create the dashboard
print("=== FINANCIAL INTELLIGENCE DASHBOARD ===")
create_dashboard()

In [ ]:
# Print dashboard summary
print("\n📊 DASHBOARD SUMMARY")
print("="*40)

if intelligence_system.portfolio:
    summary = intelligence_system.get_portfolio_summary()
    print(f"📈 Portfolio: {summary['total_companies']} companies, avg {summary['average_score']:.1f}")
else:
    print("📈 Portfolio: Empty")

agent_status = intelligence_system.agent.get_status()
print(f"🤖 Agent: {agent_status['queries']} queries, {agent_status['entities_tracked']} entities")
print(f"⚠️ Alerts: {len(intelligence_system.alerts)}")

## Module Summary

🎉 **Module 6: Agents - Complete!**

### What You Built
✅ **Basic Agent** - Memory system and tool integration  
✅ **Graph Reasoning** - Multi-hop analysis over financial networks  
✅ **Complete System** - End-to-end investment analysis with portfolio management

### Key Capabilities
- Agent memory for conversation and entity tracking
- Graph-based reasoning for competitive and risk analysis  
- Tool integration for external data sources
- Investment scoring and portfolio monitoring
- Alert generation and performance tracking

### Next Steps
🚀 **Extensions to Try:**
- Connect to real financial APIs (Alpha Vantage, SEC EDGAR)
- Add more sophisticated reasoning patterns
- Implement real-time monitoring and alerts
- Build web interface with Streamlit

📚 **Continue Learning:**
- Neo4j Graph Data Science Library
- Advanced agent frameworks (LangChain, CrewAI)
- Production deployment patterns

### Course Complete! 🏆
You've mastered all 6 modules of **Modular GenAI Workshops 2025**:
1. Graph Basics → 2. Structured Data → 3. Unstructured Data → 4. Analytics → 5. Retrievers → 6. **Agents**

**You're now equipped to build sophisticated graph-powered AI systems!**

In [ ]:
# Final system status and cleanup
print("🎊 MODULE 6: AGENTS - COMPLETED! 🎊")
print("🏆 MODULAR GENAI WORKSHOPS 2025 - COURSE COMPLETED! 🏆\n")

# Final statistics
agent_status = intelligence_system.agent.get_status()
portfolio_summary = intelligence_system.get_portfolio_summary()

print("📊 FINAL SYSTEM STATISTICS")
print("="*40)
print(f"🤖 Agent Queries: {agent_status['queries']}")
print(f"🧠 Conversations: {agent_status['conversations']}")
print(f"🏢 Entities Tracked: {agent_status['entities_tracked']}")

if 'total_companies' in portfolio_summary:
    print(f"📈 Portfolio Companies: {portfolio_summary['total_companies']}")
    print(f"📊 Average Score: {portfolio_summary['average_score']:.1f}/100")

print(f"⚠️ Total Alerts: {len(intelligence_system.alerts)}")

print(f"\n🌟 COURSE ACHIEVEMENTS")
print("="*40)
achievements = [
    "Graph Database Mastery",
    "Data Modeling & Import", 
    "NLP & Knowledge Extraction",
    "Advanced Graph Analytics",
    "GraphRAG Implementation",
    "Intelligent Agents"
]

for achievement in achievements:
    print(f"✅ {achievement}")

print(f"\n🚀 You're ready to build the future of graph-powered AI!")
print(f"📚 Continue learning: https://graphacademy.neo4j.com/")

# Optional cleanup
# driver.close()
print(f"\n✅ Session complete!")